<a href="https://colab.research.google.com/github/kvarad6/Recommendation-System-for-Workers-Customers/blob/main/Job_Recommendation_(_recommend_job_for_employees).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

In [15]:
df=pd.read_csv(r"/Final_data.csv")

In [16]:
df.head(10)

,Name,Job title,Skills,Location
0,Tanny Fomichkin,Locksmith,lock and security mechanism,Epinal
1,Clary Lowbridge,washing machine,all types of washing machines,Meitang
2,Lin Elvins,care taker,taking care of old people and small children,Luoyang
3,Cordula Gowans,web developer,design and build website,Stockholm
4,Accenture,architect,design new buildings,Donji Dobri
5,Alla Drewery,sweeper,cleaning and maintaining house,Wichit
6,kinesis management consultant Pvt.Ltd,Plumber,install and repair pipes,Yanjiang
7,Portie Bellie,barber,hair cutting,Kondoa
8,Jemima Gorst,cleaning worker,cleaning,Rczna
9,Atkins global,Locksmith,lock and security mechanism,Wujiao


In [18]:
df['Job title']=df['Job title'].str.strip()
df['Job title']=df['Job title'].str.lower()
df['Skills']=df['Skills'].str.strip()
df['Skills']=df['Skills'].str.lower()


In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfv=TfidfVectorizer(min_df=3, max_features=None,
                   strip_accents='unicode',analyzer='word',token_pattern=r'\w{1,}\S',
                   ngram_range=(1,3),
                   stop_words='english')
df['Job title']=df['Job title'].fillna('')
df['Skills']=df['Skills'].fillna('')
df['Job']=df['Job title']+df['Skills']

In [20]:
tfv_matrix=tfv.fit_transform(df['Job'])

In [21]:
tfv_matrix

<468x209 sparse matrix of type '<class 'numpy.float64'>'
	with 3429 stored elements in Compressed Sparse Row format>

In [22]:
from sklearn.metrics.pairwise import sigmoid_kernel

sig=sigmoid_kernel(tfv_matrix,tfv_matrix)

In [23]:
sig[0]

array([0.76359629, 0.76159416, 0.76159416, 0.76159416, 0.76159416,
       0.76159416, 0.76159416, 0.76159416, 0.76159416, 0.76359629,
       0.76359629, 0.76159416, 0.76159416, 0.76159416, 0.76359629,
       0.76159416, 0.76359629, 0.76159416, 0.76159416, 0.76359629,
       0.76159416, 0.76359629, 0.76159416, 0.76159416, 0.76159416,
       0.76159416, 0.76159416, 0.76159416, 0.76159416, 0.76159416,
       0.76159416, 0.76159416, 0.76159416, 0.76159416, 0.76159416,
       0.76159416, 0.76159416, 0.76159416, 0.76159416, 0.76159416,
       0.76159416, 0.76159416, 0.76159416, 0.76159416, 0.76159416,
       0.76159416, 0.76159416, 0.76159416, 0.76159416, 0.76159416,
       0.76159416, 0.76159416, 0.76159416, 0.76159416, 0.76159416,
       0.76159416, 0.76159416, 0.76159416, 0.76159416, 0.76159416,
       0.76159416, 0.76159416, 0.76159416, 0.76159416, 0.76159416,
       0.76159416, 0.76159416, 0.76159416, 0.76159416, 0.76159416,
       0.76159416, 0.76159416, 0.76159416, 0.76159416, 0.76159

In [24]:
ind=pd.Series(df['Job title']).drop_duplicates(keep='first')
print(ind.index)

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,  11,  12,  13,  18,
             20,  23,  24,  27,  31,  32,  42,  45,  50,  77, 108, 124, 138,
            146, 151, 155, 176, 185, 197, 212, 221, 227, 238, 260, 263, 264,
            281, 291, 316, 320, 373, 378, 383, 387, 445, 463],
           dtype='int64')


In [25]:
index_=ind
indices=pd.Series(ind.index)
indices.index=index_

In [26]:
indices

Job title
locksmith                      0
washing machine                1
care taker                     2
web developer                  3
architect                      4
sweeper                        5
plumber                        6
barber                         7
cleaning worker                8
app developer                 11
pantry cook                   12
taxi driver                   13
shop worker                   18
painter                       20
truck driver                  23
welding                       24
beautician                    27
accountant                    31
truck                         32
tester                        42
waiter                        45
housekeeper                   50
electrician                   77
baker                        108
line cook                    124
butcher cook                 138
school bus truck             146
mechanic                     151
fish cook                    155
broiler cook                 176


In [27]:
def give_rec(title , sig=sig):
    try:
        idx=indices[title]
        sig_scores=list(enumerate(sig[idx]))
        sig_scores=sorted(sig_scores, key=lambda x: x[1],reverse=True)
        sig_scores=sig_scores[1:11]
        customer_indices =[i[0] for i in sig_scores]
        return df.iloc[customer_indices]
        #values.sort_values(by = 'Experience' ,ascending=False)
       # return df.iloc[customer_indices]
        #return values.sort_values(by = 'Experience' ,ascending=False)
    except:
        contain_values=df[df['Job'].str.contains(title)]
        id=contain_values.index
        df.loc[id]
        if len(id)<15 :
            for j in range(0,len(id)): 
                if j==0:
                    idx=id[j]
                    sig_scores2=list(enumerate(sig[idx]))
                    sig_scores2=sorted(sig_scores2, key=lambda x: x[1],reverse=True)
                    sig_scores=sig_scores2[1:3]
                else:
                    idx=id[j]
                    sig_scores2=list(enumerate(sig[idx]))
                    sig_scores2=sorted(sig_scores2, key=lambda x: x[1],reverse=True)
                    sig_scores=sig_scores+sig_scores2[1:3]
        else :
             for j in range(5,15): 
                if j==5:
                    idx=id[j]
                    sig_scores2=list(enumerate(sig[idx]))
                    sig_scores2=sorted(sig_scores2, key=lambda x: x[1],reverse=True)
                    sig_scores=sig_scores2[1:3]
                else:
                    idx=id[j]
                    sig_scores2=list(enumerate(sig[idx]))
                    sig_scores2=sorted(sig_scores2, key=lambda x: x[1],reverse=True)
                    sig_scores=sig_scores+sig_scores2[1:3]
        customer_indices=[i[0] for i in sig_scores]
        print(df.iloc[customer_indices])
       # print(value.sort_values(by = 'Experience' ,ascending=False))

In [28]:
give_rec('cleaning')

                            Name  ...                      Job
22             Carmelita Hadaway  ...  cleaning workercleaning
29  Agile HR consultancy Pvt.Ltd  ...  cleaning workercleaning
22             Carmelita Hadaway  ...  cleaning workercleaning
29  Agile HR consultancy Pvt.Ltd  ...  cleaning workercleaning
22             Carmelita Hadaway  ...  cleaning workercleaning
29  Agile HR consultancy Pvt.Ltd  ...  cleaning workercleaning
22             Carmelita Hadaway  ...  cleaning workercleaning
29  Agile HR consultancy Pvt.Ltd  ...  cleaning workercleaning
22             Carmelita Hadaway  ...  cleaning workercleaning
29  Agile HR consultancy Pvt.Ltd  ...  cleaning workercleaning
22             Carmelita Hadaway  ...  cleaning workercleaning
29  Agile HR consultancy Pvt.Ltd  ...  cleaning workercleaning
22             Carmelita Hadaway  ...  cleaning workercleaning
29  Agile HR consultancy Pvt.Ltd  ...  cleaning workercleaning
22             Carmelita Hadaway  ...  cleaning workerc

In [29]:
give_rec('hair cutting')

             Name Job title        Skills         Location                 Job
17  Ardene Widdop    barber  hair cutting            Dosso  barberhair cutting
25      Accenture    barber  hair cutting  Skinnskatteberg  barberhair cutting
17  Ardene Widdop    barber  hair cutting            Dosso  barberhair cutting
25      Accenture    barber  hair cutting  Skinnskatteberg  barberhair cutting
17  Ardene Widdop    barber  hair cutting            Dosso  barberhair cutting
25      Accenture    barber  hair cutting  Skinnskatteberg  barberhair cutting
17  Ardene Widdop    barber  hair cutting            Dosso  barberhair cutting
25      Accenture    barber  hair cutting  Skinnskatteberg  barberhair cutting
17  Ardene Widdop    barber  hair cutting            Dosso  barberhair cutting
25      Accenture    barber  hair cutting  Skinnskatteberg  barberhair cutting
17  Ardene Widdop    barber  hair cutting            Dosso  barberhair cutting
25      Accenture    barber  hair cutting  Skinnskat

In [30]:
give_rec('working in various shops')

              Name    Job title  ...  Location                                  Job
47  Marty Attewill  shop worker  ...  Xiaotang  shop workerworking in various shops
57    Sinai global  shop worker  ...   Guchang  shop workerworking in various shops
47  Marty Attewill  shop worker  ...  Xiaotang  shop workerworking in various shops
57    Sinai global  shop worker  ...   Guchang  shop workerworking in various shops
47  Marty Attewill  shop worker  ...  Xiaotang  shop workerworking in various shops
57    Sinai global  shop worker  ...   Guchang  shop workerworking in various shops
47  Marty Attewill  shop worker  ...  Xiaotang  shop workerworking in various shops
57    Sinai global  shop worker  ...   Guchang  shop workerworking in various shops
47  Marty Attewill  shop worker  ...  Xiaotang  shop workerworking in various shops
57    Sinai global  shop worker  ...   Guchang  shop workerworking in various shops
47  Marty Attewill  shop worker  ...  Xiaotang  shop workerworking in variou

In [31]:
give_rec('accounting')

       Name   Job title      Skills   Location                   Job
43  premium  accountant  accounting        Ore  accountantaccounting
72  premium  accountant  accounting  Goyang-si  accountantaccounting
43  premium  accountant  accounting        Ore  accountantaccounting
72  premium  accountant  accounting  Goyang-si  accountantaccounting
43  premium  accountant  accounting        Ore  accountantaccounting
72  premium  accountant  accounting  Goyang-si  accountantaccounting
43  premium  accountant  accounting        Ore  accountantaccounting
72  premium  accountant  accounting  Goyang-si  accountantaccounting
43  premium  accountant  accounting        Ore  accountantaccounting
72  premium  accountant  accounting  Goyang-si  accountantaccounting
43  premium  accountant  accounting        Ore  accountantaccounting
72  premium  accountant  accounting  Goyang-si  accountantaccounting
43  premium  accountant  accounting        Ore  accountantaccounting
72  premium  accountant  accountin

In [32]:
give_rec('lock and security mechanism')

              Name  Job title  ...    Location                                   Job
9    Atkins global  locksmith  ...      Wujiao  locksmithlock and security mechanism
10  Adolph Dagnall  locksmith  ...  Tanumshede  locksmithlock and security mechanism
9    Atkins global  locksmith  ...      Wujiao  locksmithlock and security mechanism
10  Adolph Dagnall  locksmith  ...  Tanumshede  locksmithlock and security mechanism
9    Atkins global  locksmith  ...      Wujiao  locksmithlock and security mechanism
10  Adolph Dagnall  locksmith  ...  Tanumshede  locksmithlock and security mechanism
9    Atkins global  locksmith  ...      Wujiao  locksmithlock and security mechanism
10  Adolph Dagnall  locksmith  ...  Tanumshede  locksmithlock and security mechanism
9    Atkins global  locksmith  ...      Wujiao  locksmithlock and security mechanism
10  Adolph Dagnall  locksmith  ...  Tanumshede  locksmithlock and security mechanism
9    Atkins global  locksmith  ...      Wujiao  locksmithlock and

In [33]:
give_rec('taking care of old people and small children')

                 Name  ...                                                Job
34      Atkins global  ...  care takertaking care of old people and small ...
41  Dorothy Scatchard  ...  care takertaking care of old people and small ...
34      Atkins global  ...  care takertaking care of old people and small ...
41  Dorothy Scatchard  ...  care takertaking care of old people and small ...
34      Atkins global  ...  care takertaking care of old people and small ...
41  Dorothy Scatchard  ...  care takertaking care of old people and small ...
34      Atkins global  ...  care takertaking care of old people and small ...
41  Dorothy Scatchard  ...  care takertaking care of old people and small ...
34      Atkins global  ...  care takertaking care of old people and small ...
41  Dorothy Scatchard  ...  care takertaking care of old people and small ...
34      Atkins global  ...  care takertaking care of old people and small ...
41  Dorothy Scatchard  ...  care takertaking care of old people 

In [34]:
give_rec('excellent knowledge of motor technology')

              Name  ...                                              Job
156  Atkins global  ...  mechanicexcellent knowledge of motor technology
157    Kim Cattini  ...  mechanicexcellent knowledge of motor technology
156  Atkins global  ...  mechanicexcellent knowledge of motor technology
157    Kim Cattini  ...  mechanicexcellent knowledge of motor technology
156  Atkins global  ...  mechanicexcellent knowledge of motor technology
157    Kim Cattini  ...  mechanicexcellent knowledge of motor technology
156  Atkins global  ...  mechanicexcellent knowledge of motor technology
157    Kim Cattini  ...  mechanicexcellent knowledge of motor technology
156  Atkins global  ...  mechanicexcellent knowledge of motor technology
157    Kim Cattini  ...  mechanicexcellent knowledge of motor technology
156  Atkins global  ...  mechanicexcellent knowledge of motor technology
157    Kim Cattini  ...  mechanicexcellent knowledge of motor technology
156  Atkins global  ...  mechanicexcellent knowledg